In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import category_encoders as ce
import functools

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Conv1D, Embedding,Reshape, Concatenate,BatchNormalization,GlobalMaxPooling1D,GlobalAveragePooling1D
# from keras.layers.merge import Concatenate
from keras.preprocessing import sequence
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras import backend as K
# from keras.wrappers.scikit_learn import KerasClassifier
from keras import metrics
from keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction import FeatureHasher

# Lectura de datos

In [2]:
data = pd.read_csv('C:/Users/ASUS/Documents/Data mining/splited_RASFF_DATA_16092019.csv', sep=';', 
                header=0, index_col = 0)
data ['DATE_CASE'] = data ['DATE_CASE'].astype(str)
data ['HAZARDS_CAT'] = data ['HAZARDS_CAT'].astype(str)
data = data.sample(frac = 1)
df = data
df

,DATE_CASE,NOT_COUNTRY,PROD_CAT,TYPE,RISK_DECISION,ACTION_TAKEN,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN,COUNT_DESTIN,COUNT_CONCERN
10466,2018-12-14,Netherlands,fish and fish products,food,serious,informing recipient(s),product (presumably) no longer on the market,biological contaminants (other),Maldives,Sweden,Netherlands
33745,2017-07-07,United Kingdom,confectionery,food,undecided,withdrawal from the market,distribution to other member countries,labelling absent/incomplete/incorrect,United States,France,
20087,2018-05-07,Germany,feed materials,feed,serious,withdrawal from the market,distribution to other member countries,composition,Ukraine,Slovenia,INFOSAN
78089,2013-02-08,United Kingdom,prepared dishes and snacks,food,not serious,recall from consumers,distribution to other member countries,adulteration / fraud,Romania,Portugal,Slovenia
9951,2018-12-27,Germany,"soups, broths, sauces and condiments",food,serious,detained by operator,distribution to other member countries,food additives and flavourings,Italy,Lithuania,Commission Services
...,...,...,...,...,...,...,...,...,...,...,...
22172,2018-03-16,Netherlands,"nuts, nut products and seeds",food,serious,public warning - press release,distribution to other member countries,pathogenic micro-organisms,India,Germany,Slovakia
72853,2013-07-29,Netherlands,alcoholic beverages,food,serious,recall from consumers,distribution to other member countries,packaging defective / incorrect,Belgium,Switzerland,
54019,2015-07-22,Germany,feed materials,feed,serious,informing authorities,no distribution from notifying country,environmental pollutants,Poland,Germany,Austria
10516,2018-12-11,Sweden,"dietetic foods, food supplements, fortified foods",food,serious,,product traded online,composition,United States,,INFOSAN


In [3]:
mask = (df['DATE_CASE'] >= '2004-01-01') & (df['DATE_CASE'] <= '2018-12-31')
df = df.loc[mask]
df['DATE_CASE'] =pd.to_datetime(df.DATE_CASE)
df['DATE_CASE'] = df.DATE_CASE.dt.month

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4436\335135510.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_CASE'] =pd.to_datetime(df.DATE_CASE)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4436\335135510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_CASE'] = df.DATE_CASE.dt.month


In [4]:
features = [0,1,2,6,7,8]
target = [5]
X = df.iloc[:,features]
y = df.iloc[:,target]

In [5]:
X

,DATE_CASE,NOT_COUNTRY,PROD_CAT,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN
10466,12,Netherlands,fish and fish products,product (presumably) no longer on the market,biological contaminants (other),Maldives
33745,7,United Kingdom,confectionery,distribution to other member countries,labelling absent/incomplete/incorrect,United States
20087,5,Germany,feed materials,distribution to other member countries,composition,Ukraine
78089,2,United Kingdom,prepared dishes and snacks,distribution to other member countries,adulteration / fraud,Romania
9951,12,Germany,"soups, broths, sauces and condiments",distribution to other member countries,food additives and flavourings,Italy
...,...,...,...,...,...,...
22172,3,Netherlands,"nuts, nut products and seeds",distribution to other member countries,pathogenic micro-organisms,India
72853,7,Netherlands,alcoholic beverages,distribution to other member countries,packaging defective / incorrect,Belgium
54019,7,Germany,feed materials,no distribution from notifying country,environmental pollutants,Poland
10516,12,Sweden,"dietetic foods, food supplements, fortified foods",product traded online,composition,United States


In [6]:
y

,ACTION_TAKEN
10466,informing recipient(s)
33745,withdrawal from the market
20087,withdrawal from the market
78089,recall from consumers
9951,detained by operator
...,...
22172,public warning - press release
72853,recall from consumers
54019,informing authorities
10516,


# Selección de codificación

In [7]:
ency = OneHotEncoder(handle_unknown='ignore', sparse_output=False)


# Fit the encoder to your data

ency.fit(y.values)


# Transform the data to one-hot encoded format

y_one_hot = ency.transform(y.values)

In [8]:
'''encx = OrdinalEncoder()
encx.fit(X.values)
x_one_hot = encx.transform(X.values)'''

'encx = OrdinalEncoder()\nencx.fit(X.values)\nx_one_hot = encx.transform(X.values)'

In [9]:
'''encx = OneHotEncoder(handle_unknown='ignore', sparse = False)
encx.fit(X.values)
x_one_hot = encx.transform(X.values)'''

"encx = OneHotEncoder(handle_unknown='ignore', sparse = False)\nencx.fit(X.values)\nx_one_hot = encx.transform(X.values)"

In [10]:
encx  = ce.BaseNEncoder()
encx.fit(X.values)
x_one_hot = encx.transform(X.values)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(x_one_hot, y_one_hot, test_size=0.2)

# Pruebas con mlp básico

In [12]:
accs = []
for i in range (1,6):
    K.clear_session()
    model = Sequential()
    model.add(Dense(2048, activation="relu", input_dim=len(X_train.values[1])))
    model.add(Dropout(0.3))
    model.add(Dense(1024, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(24, activation = "softmax"))
    model.compile(loss='categorical_crossentropy', optimizer="adam",  metrics=['categorical_accuracy'])
    hist = model.fit(X_train,y_train, epochs= 15,validation_data=(X_val, y_val), batch_size = 64)
    accs.append(hist.history['val_categorical_accuracy'][9])
    model.save ("model"+str(i)+".h5")


Epoch 1/15


C:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1647/1647 ━━━━━━━━━━━━━━━━━━━━ 74s 43ms/step - categorical_accuracy: 0.4221 - loss: 1.9102 - val_categorical_accuracy: 0.5870 - val_loss: 1.3585
Epoch 2/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 75s 45ms/step - categorical_accuracy: 0.5944 - loss: 1.3233 - val_categorical_accuracy: 0.6544 - val_loss: 1.1405
Epoch 3/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 73s 44ms/step - categorical_accuracy: 0.6518 - loss: 1.1174 - val_categorical_accuracy: 0.6876 - val_loss: 1.0108
Epoch 4/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 76s 46ms/step - categorical_accuracy: 0.6892 - loss: 0.9790 - val_categorical_accuracy: 0.7082 - val_loss: 0.9380
Epoch 5/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 76s 46ms/step - categorical_accuracy: 0.7160 - loss: 0.8806 - val_categorical_accuracy: 0.7282 - val_loss: 0.8910
Epoch 6/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 77s 47ms/step - categorical_accuracy: 0.7396 - loss: 0.8092 - val_categorical_accuracy: 0.7425 - val_loss: 0.8500
Epoch 7/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 59s 36ms/step - categorical_accur

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1647/1647 ━━━━━━━━━━━━━━━━━━━━ 88s 53ms/step - categorical_accuracy: 0.8025 - loss: 0.5768 - val_categorical_accuracy: 0.7773 - val_loss: 0.7835
Epoch 14/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 85s 52ms/step - categorical_accuracy: 0.8088 - loss: 0.5587 - val_categorical_accuracy: 0.7763 - val_loss: 0.7853
Epoch 15/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 136s 48ms/step - categorical_accuracy: 0.8108 - loss: 0.5515 - val_categorical_accuracy: 0.7814 - val_loss: 0.7860


C:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 108s 51ms/step - categorical_accuracy: 0.4258 - loss: 1.9026 - val_categorical_accuracy: 0.5800 - val_loss: 1.3605
Epoch 2/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 78s 47ms/step - categorical_accuracy: 0.5956 - loss: 1.3145 - val_categorical_accuracy: 0.6521 - val_loss: 1.1257
Epoch 3/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 74s 45ms/step - categorical_accuracy: 0.6547 - loss: 1.1019 - val_categorical_accuracy: 0.6913 - val_loss: 1.0068
Epoch 4/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 74s 45ms/step - categorical_accuracy: 0.6934 - loss: 0.9684 - val_categorical_accuracy: 0.7113 - val_loss: 0.9331
Epoch 5/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 80s 49ms/step - categorical_accuracy: 0.7154 - loss: 0.8801 - val_categorical_accuracy: 0.7271 - val_loss: 0.8887
Epoch 6/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 78s 47ms/step - categorical_accuracy: 0.7371 - loss: 0.8087 - val_categorical_accuracy: 0.7388 - val_loss: 0.8543
Epoch 7/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 78s 47ms/step - categ

C:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 64s 34ms/step - categorical_accuracy: 0.4237 - loss: 1.9078 - val_categorical_accuracy: 0.5840 - val_loss: 1.3584
Epoch 2/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 53s 32ms/step - categorical_accuracy: 0.5907 - loss: 1.3253 - val_categorical_accuracy: 0.6518 - val_loss: 1.1319
Epoch 3/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 53s 32ms/step - categorical_accuracy: 0.6504 - loss: 1.1123 - val_categorical_accuracy: 0.6871 - val_loss: 1.0162
Epoch 4/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 54s 33ms/step - categorical_accuracy: 0.6877 - loss: 0.9819 - val_categorical_accuracy: 0.7077 - val_loss: 0.9405
Epoch 5/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 54s 33ms/step - categorical_accuracy: 0.7135 - loss: 0.8969 - val_categorical_accuracy: 0.7277 - val_loss: 0.8872
Epoch 6/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 53s 32ms/step - categorical_accuracy: 0.7340 - loss: 0.8150 - val_categorical_accuracy: 0.7406 - val_loss: 0.8506
Epoch 7/15
1647/1647 ━━━━━━━━━━━━━━━━━━━━ 53s 32ms/step - catego

In [13]:
suma = 0
for i in accs:
    suma = suma + i
print(suma/5)

0.7666653990745544
